---
title: "INDY Notes & Simulations"
format: dashboard
server: shiny
editor:
  markdown:
    wrap: 72
---

In [ ]:
#| echo: false

# format: dashboard
# server: shiny
# editor: 
#   markdown: 
#     wrap: 72

# ---
# title: "INDY Notes & Simulations"
# format: html
# execute:
#   eval: false
# ---

############################################################################################################ 

# § Percolation Demo


In [ ]:
#| echo: false


import shiny
import random
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib.lines import Line2D


import numpy as np
from numpy.random import poisson, uniform

import scipy.sparse

from sklearn.cluster import SpectralClustering
from sklearn.metrics import adjusted_rand_score


from shiny.express import render, ui

In [ ]:
#| label: graph_builder.py

import graphlearning as gl

def get_Gaussian_weight_matrix(X, k):
    Z = gl.weightmatrix.knn(X, k)  # Gaussian similarity measure
    A = (Z + Z.T) / 2
    return A

In [ ]:
#| label: metric_backbone.py

import networkx as nx
import igraph as ig

def get_metric_backbone_igraph(D):
    """
     :param D: networkx distance graph (with weight and proximity edge attribute)
     :return: Networkx Metric Backbone subgraph of D
    """
    D_ig = ig.Graph.from_networkx(D)
    #print(D_ig.distances(weights='weight'))
    distances = D_ig.distances(weights='weight')

    B = nx.Graph(D)
    B.remove_edges_from([(x, y) for x, y, w in B.edges.data('weight') if w > distances[x][y]])
    return B

In [ ]:
#| label: global-variables

global_SEED = 42

##  {.sidebar width="300px"}


In [ ]:
ui.input_select("p", "Probability:",
                choices=[x / 100 for x in range(1, 100)],
                selected=0.5,
                width=10
)
ui.input_select("grid_size", "Number of nodes in each dimension:",
                choices=[x for x in range(2, 101)],
                selected=10
)

## Column


In [ ]:
@render.plot
def displot():
    p = float(input.p())
    grid_size = int(input.grid_size())
    G = nx.grid_2d_graph(grid_size, grid_size)
    pos = {(x, y): (x, y) for x, y in G.nodes()}
    
    plt.figure(figsize=(6, 6))
    
    for (u, v) in G.edges():
        edge_color = 'red' if random.random() < p else 'black'
        edge_width = 3 if edge_color == 'red' else 1  # Thicker for red edges
        nx.draw_networkx_edges(G, pos, edgelist=[(u, v)], edge_color=edge_color, width=edge_width)
    
    nx.draw_networkx_nodes(G, pos, node_size=0)
    
    legend_elements = [
        Line2D([0], [0], color='red', lw=3, label  ='open   & w(e) = 1'),
        Line2D([0], [0], color='black', lw=1, label='closed & w(e) = 0'),
    ]
    
    plt.legend(handles=legend_elements, bbox_to_anchor=(1.4, 0.96))
    plt.gca().set_aspect('equal')
    plt.axis('off')
    plt.title(f"{grid_size}x{grid_size} Grid with p = {p:.2f}", fontsize=14)
    plt.text(0.5, -0.05, 'Figure 1: each edge is open with probability p.',
         fontsize=12, ha='center', va='center', transform=plt.gca().transAxes)

    # Adjust layout to ensure the caption fits within the figure area
    plt.tight_layout()

############################################################################################################ 

# § Introduction

<h2>Bernoulli Bond Percolation on $\mathbb{Z}^d$</h2>

<h3>Setup:</h3>

-   $G = (\mathbb{Z}^d, E)$
-   $E = \{(x, y) \in \mathbb{Z}^d \times \mathbb{Z}^d : ||x - y||_1 = 1\}$

<h3>Probabilistic framework $(\Omega, \mathcal{F}, \mathbb{P_p})$</h3>

-   $\Omega = \{0, 1\}^E$
-   $\mathcal{F}=$ product $\sigma$-algebra
-   $\mathbb{P}_p = \text{Bernoulli}(p)^{\bigotimes E}$

<h3>Properties:</h3>

-   Let $\omega, \eta \in \{0, 1\}^E$ be two configurations. We define
    the following **ordering**: <br>

$\qquad \qquad \omega \leq \eta \quad \iff \quad \omega(e) \leq \eta(e) \quad \forall e \in E.$
<br>

-   An **event** $A \in \mathcal{F}$ is **increasing** if

    $\qquad \omega \in A \quad \& \quad \omega \leq \eta \implies \eta \in A.$
    <br>

    -   Examples:
        $\{x \xleftrightarrow{} y\} \text{ } \& \text{ } \{|C_x| \geq 10\}$
        are increasing events.

    -   Remark : $A, B$ increasing
        $\implies A \cap B \text{ } \& \text{ } A \cup B$ increasing.

    -   Nonexample: $\{|C_0| = 10\}$ is not increasing. <br> <br>

-   A **function** $f: \Omega \to \mathbb{R}$ is **increasing** if

    $\qquad \omega \leq \eta \implies f(\omega) \leq f(\eta).$ <br>

    -   Example: $f(\omega) = |C_0(\omega)|$ is an increasing function.

    -   Connection: event $A$ increasing $\iff$ function $\mathbb{1}_A$
        increasing. <br> <br>

-   Les $(\Omega, \mathcal{A}, P)$ be a probability space. <br> A map
    $X: \Omega \to \mathbb{R}$ is a **random variable** if
    $X^{-1}(B) \stackrel{\text{notation}}{=} \{X \in B\} \in \mathcal{A}$
    for all Borel sets $B \in \mathcal{B}(\mathbb{R})$. <br> In other
    words: $X$ must a measurable map from $(\Omega, \mathcal{A})$ to
    $(\mathbb{R}, \mathcal{B}(\mathbb{R}))$. Note that the probability
    measure is irrelevant here. <br> <br> Each random variable induces a
    probability measure on $\mathbb{R}$ by
    $P_X(B) = P(X \in B) = P(X^{-1}(B))$ called the **law** of $X$. <br>
    <br>

-   **Proposition:**

    1.  Let $A \in \mathcal{F}$ be an increasing event, then <br>
        $\qquad p \mapsto \mathbb{P}_p(A)$ is non-decreasing. <br> <br>
    2.  Let $f: \Omega \to \mathbb{R}$ be a measurable, increasing,
        nonnegative, bounded function. Then <br>
        $\qquad p \mapsto \mathbb{E}_p[f]$ is non-decreasing. <br> <br>

    -   Proof:

        -   

            2.  $\implies$ 1. by taking $f = \mathbb{1}_A$.

        -   Define $X_p(e) = \mathbb{1}_{\{U_e \leq p\}}$ so that <br>
            $\qquad E[f(X_p)] = E[f(\mathbb{1}_{\{U_e \leq p\}})] = p \cdot f(1) + (1 - p) \cdot f(0)$
            <br>

        -   $P(X_p(e) = 1) = p$ and $P(X_p(e) = 0) = 1 - p$ so that the
            law associated to the $X_p$s is the same as the one
            associated to $\mathbb{P}_p$. <br>

        -   Thus, the expectation of $f$ under the law of $X_p$ is the
            same as the expectation of $f$ under $\mathbb{P}_p$. Indeed,
            the change of variable formula gives <br>
            $\qquad E_P[g(Y)] = \int_{\Omega} g \circ Y dP = \int_{\mathbb{R}} g d \mu_{\text{pushforward of P with respect to Y}} = \int_{\mathbb{R}} g dP_Y$,
            where <br>

            $\qquad P_Y(B) = P(Y \in B) = P(Y^{-1}(B))$ is the law of
            $Y$. <br>

            In our case, this gives <br>

            $\qquad E[f(X_p)] = \int_{\mathbb{R}} f dP_{X_p} \quad (g \to f, \text{ } Y \to X_p)$
            which is equal to <br>

        -   In the context of the proof, we have $Y = X_p$ and $g = f$
            so that <br>
            $\qquad E[f(X_p)] = E_P[f(X_p)] = E_{\mathbb{P}_p}[f] = E_p[f]$
            <br>

############################################################################################################ 

# § Sheet 1

<h2>Exercise 1</h2>

$\{A \xleftrightarrow{} B\}$ is measurable
$\forall A, B \subset \mathbb{Z}^d$, and the function below is
measurable.


```{=tex}
\begin{aligned}
F: \quad & \{0, 1\}^E & \to     & \quad N \cup \{\infty\} \\
         & \omega     & \mapsto & \quad \text{size of the cluster containing the origin} = |C_0(\omega)|.
\end{aligned}
```

<h2>Exercise 2</h2>

The event $\{0 \xleftrightarrow{} \infty\}$ is measurable.

<h2>Exercise 3</h2>

Compute the following probabilities:
$\quad P_p(|C_0| = 0), \quad P_p(|C_0| = 1), \quad P_p(|C_0| \geq 1) \quad \& \quad P_p(|C_0| \geq 1 \big{|} |C_x| = 0) \text{ for some } x \in \mathbb{Z}^d.$

<h2>Exercise 4</h2>

On $\mathbb{Z}^2$, consider the event
$$C_{2n,n} = \{\exists \text{ open path from left to right in the box } [0, 2n] \times [0, n]\}$$

Let $q_n = 1 - P_p(C_{2n,n})$. Show that one of the following holds:

-   $\exists \varepsilon > 0$ such that $q_n \geq \varepsilon$ for all
    $n$.
-   $\exists \varepsilon > 0$ such that $q_n \leq e^{- \varepsilon n}$
    for all $n$.

Based on this result, prove that $p_c < 1$.

############################################################################################################ 

# § Metric Backbone & Spectral Clustering Simulations


##  {.sidebar width="300px"}


In [ ]:
np.random.seed(42)

input_select_width = 10

L = list(range(100, 501, 100))
L.insert(0, 50)
# L.append(50)

ui.input_select("n", "Number of nodes in each cluster:",
                choices=L,
                selected=50,
                width=input_select_width
)

ui.input_select("d", "Number of dimensions & communities:",
                choices=list((2, 3, 4)),
                selected=2,
                width=input_select_width
)

ui.input_select("k", "Number of nearest neighbors for graph creation:",
                choices=list((5, 10, 15)),
                selected=10,
                width=input_select_width
)

ui.input_select("n_neighbors", "Number of nearest neighbors for spectral clustering:",
                choices=list(range(3, 16)),
                selected=4,
                width=input_select_width
)

ui.input_select("mu_x2", "Mean of the second Gaussian with respect to the x-axis:",
                choices=list(range(1, 21)),
                selected=3,
                width=input_select_width
)


ui.input_select("λ", "Intensity parameter (N_n ~ Poisson(λ * n)):",
                choices=[1],
                selected=1,
                width=input_select_width
)

ui.input_select("R_1", "Big radius for intra-community edges:",
                choices=list(range(1, 11)),
                selected=3,
                width=input_select_width
)

ui.input_select("R_2", "Small radius for inter-community edges:",
                choices=[1, 1.5, 2, 2.5, 3],
                selected=[1.5],
                width=input_select_width
)

## Column


In [ ]:
#| label: community-detection-on-Euclidean-graphs-FUNCTIONS

def euclidean_distance(tuple_1, tuple_2):
    return np.linalg.norm(np.array(tuple_1) - np.array(tuple_2))


def φ(R):
    return lambda r: 1 if r <= R else 0


def f(f_r):
    return lambda G_distance, u_idx, v_idx: f_r(euclidean_distance(tuple_1=G_distance.nodes[u_idx]['pos'], tuple_2=G_distance.nodes[v_idx]['pos']))

In [ ]:
#| label: Inter- & Intra-Community Proportion Functions


def get_inter_proportion(G):
  
  nominator = 0
  
  for u, v in G.edges():
    if G.nodes[u]['community'] != G.nodes[v]['community']:
      nominator += 1
      
  denominator = G.number_of_edges()
      
  res = nominator / denominator
  
  return res


def get_intra_proportion(G):
  return 1 - get_inter_proportion(G)

In [ ]:
#| label: functions to produce samples

 
# def produce_gaussian_samples(n, d, mu_x2, SEED=global_SEED):
# 
#     n_rows = n * d
#     samples = np.empty((n_rows, d + 1))
#     samples[:, 0] = np.arange(len(samples))
# 
#     col_slice = slice(1, samples.shape[1] + 1)
# 
#     rng = np.random.default_rng(42)
#     idx = 0
#     for last_row in range(0, n_rows, n):  # step size is n
#         mean_val = mu_x2 * idx
#         row_slice = slice(last_row, last_row + n)
#         samples[row_slice, col_slice] = rng.multivariate_normal(
#             mean=np.insert(np.zeros(d - 1), 0, mean_val), cov=np.eye(d), size=n
#         )
#         idx += 1
# 
#     return samples
# 
# 
# def produce_uniform_samples(n, d, SEED=global_SEED):
# 
#     n_nodes = n * d
#     d_root_n = n_nodes ** (1 / d)
# 
#     samples        = np.empty((n_nodes, d + 1))
#     samples[:, 0]  = np.arange(len(samples))
#     
#     rng = np.random.default_rng(SEED)
#     samples[:, 1:] = d_root_n  * rng.uniform(size=(n_nodes, d))  # type(.)    : np.ndarray
#                                                                  # np.shape(.): (N_n, d)
#     return samples
  

def produce_samples(n, d , type_samples, mu_x2=None, SEED=global_SEED):
    
    rng = np.random.default_rng(42)
  
    n_rows = n * d  # one row per node
    samples = np.empty((n_rows, d + 1))
    samples[:, 0] = np.arange(len(samples))
    
    
    if type_samples == "gaussian":
    
        col_slice = slice(1, samples.shape[1] + 1)
    
        idx = 0
        for last_row in range(0, n_rows, n):  # step size is n
            mean_val = mu_x2 * idx
            row_slice = slice(last_row, last_row + n)
            samples[row_slice, col_slice] = rng.multivariate_normal(
                mean=np.insert(np.zeros(d - 1), 0, mean_val), cov=np.eye(d), size=n
            )
            idx += 1
    
    
    elif type_samples == "uniform":
      
        samples[:, 1:] = d_root_n  * rng.uniform(size=(n_nodes, d))  # type(.)    : np.ndarray
                                                                     # np.shape(.): (N_n, d)
      
    else:
        raise ValueError("type_samples must be either 'gaussian' or 'uniform'")
      
    
    return samples

In [ ]:
#| label: functions to produce distance graphs


def produce_G_distance(n, k, samples):
    
    G_distance = nx.Graph()

    d = {int(row[0]): (row[1], row[2]) for row in samples}

    for node, coord in d.items():
        community = 0
            
        if node + 1 > n:
            community = 1
        
        G_distance.add_node(node, pos=coord, community=community)



    col_slice = slice(1, samples.shape[1] + 1)

    W = get_Gaussian_weight_matrix(samples[:, col_slice], k)

    for i in range(2 * n):
        for j in range(i + 1, 2 * n):
            w = W[i, j]
            if w > 0:
                G_distance.add_edge(i, j, weight=1 / w - 1)

    return G_distance


def produce_G_distance_ABBE(n, n_communities, k, samples, F):
    n_nodes = n * n_communities
    d = {int(row[0]): (row[1], row[2]) for row in samples}

    G_distance_ABBE = nx.Graph()
    G_distance_ABBE.add_nodes_from(d.keys())
    nx.set_node_attributes(G_distance_ABBE, d, 'pos')

    community_labels = np.array(range(1, n_communities + 1))
    rng = np.random.default_rng(42)
    nx.set_node_attributes(G_distance_ABBE, {node: rng.choice(community_labels) for node in G_distance_ABBE.nodes}, 'community')

    edges_to_add = [
      (u_idx, v_idx) for u_idx in range(n_nodes) for v_idx in range(u_idx + 1, n_nodes) if F(G_distance_ABBE, u_idx, v_idx) == 1
    ]
    
    G_distance_ABBE.add_edges_from(edges_to_add)

    return G_distance_ABBE

In [ ]:
#| label: draw function


def draw(G, MB, samples, n_neighbors, axs, n_clusters, L_idx=[0, 1], affinity='precomputed'):
  
    pos = nx.get_node_attributes(G, 'pos')  # Extract node positions
    
    true_labels = list(nx.get_node_attributes(G, 'community').values())
    true_colors = ['red' if label == true_labels[0] else 'blue' for label in true_labels]

    col_slice = slice(1, samples.shape[1] + 1)

    SC = SpectralClustering(n_clusters=n_clusters, affinity=affinity)
    
    def get_predColors_similarity(samples, col_slice, n_neighbors, true_labels, b_original=True, MB=None):
      
        A = None
        
        if b_original:
            A = get_Gaussian_weight_matrix(samples[:, col_slice], n_neighbors)
        else:
            A = nx.adjacency_matrix(MB, nodelist=[i for i in range(MB.number_of_nodes())], weight='proximity')
            A = scipy.sparse.csr_matrix(A)
            
        pred_labels = SC.fit_predict(A)
        pred_colors = ['red' if label == pred_labels[0] else 'blue' for label in pred_labels]
        similarity  = adjusted_rand_score(true_labels, pred_labels)
        
        if b_original:
            print(f"Adjusted Rand Score on Original Graph: {similarity * 100}")
        else:
            print(f"Adjusted Rand Score on MB : {similarity * 100}")
            
        return pred_colors, similarity
      
    pred_colors_original, similarity_original = get_predColors_similarity(samples, col_slice, n_neighbors, true_labels)
    pred_colors_mb, similarity_mb = get_predColors_similarity(
      samples=None, col_slice=None, n_neighbors=None, true_labels=true_labels, b_original=False, MB=MB
    )



    nx.draw(G, pos, node_color=true_colors, node_size=5, ax=axs[L_idx[0], 0], edge_color='lightgray')
    nx.draw(MB, pos, node_color=true_colors, node_size=5, ax=axs[L_idx[0], 1], edge_color='lightgray')
    
    nx.draw(G, pos, node_color=pred_colors_original, node_size=5, ax=axs[L_idx[1], 0], edge_color='lightgray')
    nx.draw(MB, pos, node_color=pred_colors_mb, node_size=5, ax=axs[L_idx[1], 1], edge_color='lightgray')
    
    return similarity_original, similarity_mb

In [ ]:
@render.plot
def normals_nNodes_dDimensions_PLOT():
    
    n           = int(input.n())
    d           = int(input.d())
    n_clusters  = d
    k           = int(input.k())
    mu_x2       = float(input.mu_x2())
    n_neighbors = int(input.n_neighbors())
    λ           = int(input.λ())

    R_1         = float(input.R_1())  
    R_2         = float(input.R_2())
    R_1, R_2 = max(R_1, R_2), min(R_1, R_2)
    f_in_r  = φ(R_1)
    f_out_r = φ(R_2)
    f_in  = f(f_in_r)
    f_out = f(f_out_r)


    def F(G_distance, u_idx, v_idx, f_in=f_in, f_out=f_out):
        if G_distance.nodes[u_idx]['community'] == G_distance.nodes[v_idx]['community']:
            return f_in(G_distance, u_idx, v_idx)
        else:
            return f_out(G_distance, u_idx, v_idx)





    λ_n = λ * n
    N_n = λ_n  # N_n = E[poisson(λ_n)], type: int


    # Generate samples separately
    samples_gaussian = produce_samples(n, d, mu_x2, type_samples="gaussian")

    # Update G_distance separately
    G_distance = produce_G_distance(n, k, samples_gaussian)
    mb_igraph = get_metric_backbone_igraph(G_distance)

    # Now handle plotting
    fig, axs = plt.subplots(4, 2, figsize=(24, 12))
    
    similarity_original, similarity_mb = draw(G_distance, mb_igraph, samples_gaussian, n_neighbors, axs, n_clusters)

    ############## ABBE ################
    
    SC = SpectralClustering(n_clusters=n_clusters, affinity='precomputed')

    samples_uniform = produce_samples(n, d, type_samples="uniform")
    G_distance_ABBE = produce_G_distance_ABBE(n, d, k, samples_uniform, F)

    col_slice = slice(1, samples_uniform.shape[1] + 1)

    W = get_Gaussian_weight_matrix(samples_uniform[:, col_slice], k)

    edges = list(G_distance_ABBE.edges())
    weights = {(u, v): 1 / W[u, v] - 1 if W[u, v] > 0 else float('inf') for u, v in edges}
    nx.set_edge_attributes(G_distance_ABBE, weights, 'weight')

    mb_igraph_ABBE = get_metric_backbone_igraph(G_distance_ABBE)

    similarity_original_ABBE, similarity_mb_ABBE = draw(G_distance_ABBE, mb_igraph_ABBE, samples_uniform, n_neighbors, axs, n_clusters, L_idx=[2, 3])

    # Helper functions to add legend
    def produce_patch_gaussian(color, mu_x):
        return plt.Line2D(
            [0],
            [0], 
            marker='o', 
            color='w', 
            markerfacecolor=color, 
            markersize=8, 
            label=f'X-mean: {round(mu_x)}'
        )


    def produce_patch_ABBE(color, plus_or_minus_one):
        return plt.Line2D(
            [0],
            [0], 
            marker='o', 
            color='w', 
            markerfacecolor=color, 
            markersize=8, 
            label=f'Community label: {plus_or_minus_one}'
        )


    for i in range(4):
        for j in range(2):
            ax = axs[i, j]
            ax.set_xlabel('X-axis')
            ax.set_ylabel('Y-axis')
            ax.axis('equal')
            ax.axis('on')
            ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
            if i < 2:
                ax.legend(handles=[produce_patch_gaussian('red', 0), produce_patch_gaussian('blue', mu_x2)])
            else:
                ax.legend(handles=[produce_patch_ABBE('red', 1), produce_patch_ABBE('blue', -1)])



    axs[0, 0].set_title(f'Gaussian Samples with {n} nodes in each cluster, inter-proportion: {get_inter_proportion(G_distance) * 100:.2f}%')
    axs[0, 1].set_title(f'Metric Backbone, inter-proportion: {get_inter_proportion(mb_igraph) * 100:.2f}%')
    
    axs[1, 0].set_title(f'SC: Gaussian Samples with {n} nodes in each cluster, ARI: {similarity_original * 100:.2f}%')
    axs[1, 1].set_title(f'SC: Metric Backbone, ARI: {similarity_mb * 100:.2f}%')
    
    
    axs[2, 0].set_title(f'ABBE original {G_distance_ABBE.number_of_edges()} edges, inter-proportion: {get_inter_proportion(G_distance_ABBE) * 100:.2f}%')
    axs[2, 1].set_title(f'ABBE MB {mb_igraph_ABBE.number_of_edges()} edges, inter-proportion: {get_inter_proportion(mb_igraph_ABBE) * 100:.2f}%')
    axs[3, 0].set_title(f'SC: ABBE original, ARI: {similarity_original_ABBE * 100:.2f}%')
    axs[3, 1].set_title(f'SC: ABBE MB, ARI: {similarity_mb_ABBE * 100:.2f}%')
    
    
    fig, axs = plt.subplots(4, 2, figsize=(24, 12))
    
    similarity_original, similarity_mb = draw(G_distance, mb_igraph, samples_gaussian, n_neighbors, axs, n_clusters)

    ############## ABBE ################
    
    SC = SpectralClustering(n_clusters=n_clusters, affinity='precomputed')

    samples_uniform = produce_uniform_samples(n, d, 42)
    G_distance_ABBE = produce_G_distance_ABBE(n, d, k, samples_uniform, F)

    col_slice = slice(1, samples_uniform.shape[1] + 1)

    W = get_Gaussian_weight_matrix(samples_uniform[:, col_slice], k)

    edges = list(G_distance_ABBE.edges())
    weights = {(u, v): 1 / W[u, v] - 1 if W[u, v] > 0 else float('inf') for u, v in edges}
    nx.set_edge_attributes(G_distance_ABBE, weights, 'weight')

    mb_igraph_ABBE = get_metric_backbone_igraph(G_distance_ABBE)

    similarity_original_ABBE, similarity_mb_ABBE = draw(G_distance_ABBE, mb_igraph_ABBE, samples_uniform, n_neighbors, axs, n_clusters, L_idx=[2, 3])

    # Helper functions to add legend
    def produce_patch_gaussian(color, mu_x):
        return plt.Line2D(
            [0],
            [0], 
            marker='o', 
            color='w', 
            markerfacecolor=color, 
            markersize=8, 
            label=f'X-mean: {round(mu_x)}'
        )


    def produce_patch_ABBE(color, plus_or_minus_one):
        return plt.Line2D(
            [0],
            [0], 
            marker='o', 
            color='w', 
            markerfacecolor=color, 
            markersize=8, 
            label=f'Community label: {plus_or_minus_one}'
        )


    for i in range(4):
        for j in range(2):
            ax = axs[i, j]
            ax.set_xlabel('X-axis')
            ax.set_ylabel('Y-axis')
            ax.axis('equal')
            ax.axis('on')
            ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
            if i < 2:
                ax.legend(handles=[produce_patch_gaussian('red', 0), produce_patch_gaussian('blue', mu_x2)])
            else:
                ax.legend(handles=[produce_patch_ABBE('red', 1), produce_patch_ABBE('blue', -1)])



    axs[0, 0].set_title(f'Gaussian Samples with {n} nodes in each cluster, inter-proportion: {get_inter_proportion(G_distance) * 100:.2f}%')
    axs[0, 1].set_title(f'Metric Backbone, inter-proportion: {get_inter_proportion(mb_igraph) * 100:.2f}%')
    
    axs[1, 0].set_title(f'SC: Gaussian Samples with {n} nodes in each cluster, ARI: {similarity_original * 100:.2f}%')
    axs[1, 1].set_title(f'SC: Metric Backbone, ARI: {similarity_mb * 100:.2f}%')
    
    
    axs[2, 0].set_title(f'ABBE original {G_distance_ABBE.number_of_edges()} edges, inter-proportion: {get_inter_proportion(G_distance_ABBE) * 100:.2f}%')
    axs[2, 1].set_title(f'ABBE MB {mb_igraph_ABBE.number_of_edges()} edges, inter-proportion: {get_inter_proportion(mb_igraph_ABBE) * 100:.2f}%')
    axs[3, 0].set_title(f'SC: ABBE original, ARI: {similarity_original_ABBE * 100:.2f}%')
    axs[3, 1].set_title(f'SC: ABBE MB, ARI: {similarity_mb_ABBE * 100:.2f}%')

## Conclusions

- Spectral clustering performs very poorly on ABBE graphs. 


# § graph


In [ ]:
#| echo: false

fig, axs = plt.subplots(2, 1, figsize=(6, 12))

n           = 100
d           = 2
n_clusters  = d
k           = 10
n_neighbors = 4
λ           = 1

mu_x2_grid  = np.arange(1, 5.1, 0.5).tolist()
res_G = []
res_MB = []

for mu_x2 in mu_x2_grid:
    samples = produce_gaussian_samples(n, d, mu_x2)
    G = produce_G_distance(n, k, samples)
    MB = get_metric_backbone_igraph(G)
    
    pos = nx.get_node_attributes(G, 'pos')  # Extract node positions
    
    true_labels = list(nx.get_node_attributes(G, 'community').values())
    true_colors = ['red' if label == true_labels[0] else 'blue' for label in true_labels]
    
    col_slice = slice(1, samples.shape[1] + 1)
    
    SC = SpectralClustering(n_clusters=n_clusters, affinity='precomputed')
    
    A = get_Gaussian_weight_matrix(samples[:, col_slice], n_neighbors)
        
    pred_labels = SC.fit_predict(A)
    pred_colors = ['red' if label == pred_labels[0] else 'blue' for label in pred_labels]
    similarity  = adjusted_rand_score(true_labels, pred_labels)
    
    res_G.append(similarity)
    
    A = nx.adjacency_matrix(MB, nodelist=[i for i in range(MB.number_of_nodes())], weight='proximity')
    A = scipy.sparse.csr_matrix(A)
    
    pred_labels = SC.fit_predict(A)
    pred_colors = ['red' if label == pred_labels[0] else 'blue' for label in pred_labels]
    similarity  = adjusted_rand_score(true_labels, pred_labels)
    
    res_MB.append(similarity)
    
ax = axs[0]    
ax.plot(mu_x2_grid, res_G)
ax.set_title('Original Graph')
ax.set_xlabel('Mean of the second Gaussian with respect to the x-axis')
ax.set_ylabel('ARI')

ax = axs[1]
ax.plot(mu_x2_grid, res_MB)
ax.set_title('Metric Backbone')
ax.set_xlabel('Mean of the second Gaussian with respect to the x-axis')
ax.set_ylabel('ARI')


# § Module Dependencies

In [ ]:
#print(f"Adjusted Rand Score on origin: {similarity * 100:.2f}%")

# Spectral Clustering (Original VS Metric Backbone)

# -   SSL: semi-supervised learning
# 
#     -   time to build the metric backbone
# 
# -   TSC: thresholding-based subspaces clustering
# 
# -   helper_plots: plot_TSC_k

```{mermaid}
flowchart LR
    A[clustering.py] --> B[community_experiments_plots.py]
    A --> C[community_experiments_tables.py]
    C --> D[helper_plots.py]
    E[metric_backbone.py] --> F[SSL.py]
    E --> G[graph_builder.py]
    G --> F
    G --> H[TSC.py]
    G --> B
    G --> C
    G --> D
    G --> I[datasets.py]
    I --> J[metrics.py]
    I --> F
    D --> F
    D --> H
    J --> H
    J --> B
    I --> B
    J --> C
    I --> C
    K[EffectiveResistanceSampling/Network.py] --> G
    J --> D
```


# § Proof of THM 2.1

## Change of variables formula from wikipedia

$$\int_{X_2} g \text{ } d(f_*\mu) = \int_{X_1} g \circ f \text{ } d\mu$$

for

•⁠ ⁠$f: (X_1, \Sigma_1) \to (X_2, \Sigma_2)$,

•⁠ ⁠$g: (X_2, \Sigma_2) \to (\mathbb{R}, \mathcal{B}_\mathbb{R})$,

•⁠ ⁠$\mu: \Sigma_1 \to [0, \infty]$ a measure,

•⁠ ⁠$f_\mu: \Sigma_2 \to [0, \infty]$ a measure defined by
$f_\mu(A) = \mu(f^{-1}(A))$, the pushforward measure of $\mu$ with
respect to $f$.

## Application to the proof of THM 2.1

•⁠ ⁠We replace the professor's $f$ by Grimmett's $N$.

We have

•⁠ ⁠$\mu: \mathcal{F} \to [0, \infty]$ is a measure on $[0, 1]^E$.

•⁠ ⁠$X: ([0, 1]^E, \mathbb{B(\cdot)})\to (\Omega, \mathcal{F})$ meausrable
$\implies$ \* $(X_1, \Sigma_1) = ([0, 1]^E$, $\mathbb{B(\cdot)}$), \*
$(X_2, \Sigma_2) = (\Omega, \mathcal{F})$, \* $f = X$, \* $g = N$,

So that:

$$\int_{\Omega} N \text{ } d(X_*\mu) = \int_{[0, 1]^E} N \circ X \text{ } d\mu$$

Clearly,

$$E[N \circ X] = \int_{[0, 1]^E} N \circ X \text{ } d\mu.$$

Furthermore, if we consider

•⁠ ⁠$P_p: \mathcal{F} \to [0, 1]$, •⁠
⁠$A = \{w(e_1) = u_1, \ldots, w(e_k) = u_k\} \in \mathcal{F}$

we have:

•⁠
⁠$P_p(A) = p^k(1-p)^{|E| - k} = [0, p]^k \times [p, 1]^{|E| - k} = \int_{[0, p]^k \times [p, 1]^{|E| - k}} d\lambda = \int_{X^{-1}(A)} d\lambda = \mu(X^{-1}(A))= X_*(\mu)(A)$.

so that we also have

•⁠
⁠$E_p[N] = \int_{\Omega} N \text{ } dP_p = \int_{\Omega} N \text{ } d(X_*\mu)$,

allowing us to conclude that

$$E_p[N] = E[N \circ X].$$

•⁠ ⁠Note that we are able to concentrate on sets such as
$A = \{w(e_1) = u_1, \ldots, w(e_k) = u_k\}$ because the measure $P_p$
is characterized by its output on such setss.

# § Instructions to deploy website

-   When preview yields the desired result in VsCode, compress the
    directory that contains "script.qmd" & "app.py" and upload to
    RStudio in posit.cloud

-   In posit.cloud's RStudio, do: script.qmd \> Run Document (you will
    not get a good result, don't worry)

-   rsconnect add --account rfua --name rfua --token
    81C1E677FB6E5544A763A83C69AF49E9 --secret
    EAtsm+UuJrsMEwDkmLppzkl0Q8fMZi9fnR3y4p+C

-   rsconnect deploy shiny -n rfua . (This can take up to three minutes)

-   Possible to develop directly from posit.cloud's RStudio by doing:

    -   script.qmd \> Run Document
    -   app.py \> Run App
    -   this makes it possible to see intermediary results.